<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Cargamos-librerias" data-toc-modified-id="Cargamos-librerias-1">Cargamos librerias</a></span><ul class="toc-item"><li><span><a href="#metricas-de-evaluacion-(sigmas)-+-funciones-de-utilidad" data-toc-modified-id="metricas-de-evaluacion-(sigmas)-+-funciones-de-utilidad-1.1">metricas de evaluacion (sigmas) + funciones de utilidad</a></span></li><li><span><a href="#Datos-de-entrenamiento!" data-toc-modified-id="Datos-de-entrenamiento!-1.2">Datos de entrenamiento!</a></span></li><li><span><a href="#preprocesamiento-para-X-y-Y" data-toc-modified-id="preprocesamiento-para-X-y-Y-1.3">preprocesamiento para X y Y</a></span></li></ul></li><li><span><a href="#Tensorflow" data-toc-modified-id="Tensorflow-2">Tensorflow</a></span></li><li><span><a href="#Keras" data-toc-modified-id="Keras-3">Keras</a></span><ul class="toc-item"><li><span><a href="#Red-sencillita" data-toc-modified-id="Red-sencillita-3.1">Red sencillita</a></span></li><li><span><a href="#Calidad-de-vida:-liveloss!" data-toc-modified-id="Calidad-de-vida:-liveloss!-3.2">Calidad de vida: liveloss!</a></span></li><li><span><a href="#Calidad-de-vida:-agregando-un-tensorboard" data-toc-modified-id="Calidad-de-vida:-agregando-un-tensorboard-3.3">Calidad de vida: agregando un tensorboard</a></span></li><li><span><a href="#Trucos:-Regularizadores-de-redes" data-toc-modified-id="Trucos:-Regularizadores-de-redes-3.4">Trucos: Regularizadores de redes</a></span></li><li><span><a href="#Actividad:-Crea-una-red-profunda" data-toc-modified-id="Actividad:-Crea-una-red-profunda-3.5">Actividad: Crea una red profunda</a></span></li><li><span><a href="#trucos:-Dropout" data-toc-modified-id="trucos:-Dropout-3.6">trucos: Dropout</a></span></li><li><span><a href="#Trucos:-BatchNormalization-y-Activacion!" data-toc-modified-id="Trucos:-BatchNormalization-y-Activacion!-3.7">Trucos: BatchNormalization y Activacion!</a></span></li><li><span><a href="#Keras-Functional" data-toc-modified-id="Keras-Functional-3.8">Keras Functional</a></span></li><li><span><a href="#Truco:-Skip-connections" data-toc-modified-id="Truco:-Skip-connections-3.9">Truco: Skip connections</a></span></li><li><span><a href="#Optimizador" data-toc-modified-id="Optimizador-3.10">Optimizador</a></span></li><li><span><a href="#Controlando-el-learning-rate" data-toc-modified-id="Controlando-el-learning-rate-3.11">Controlando el learning rate</a></span></li><li><span><a href="#Actividad:-Crea-una-red-poderosa!" data-toc-modified-id="Actividad:-Crea-una-red-poderosa!-3.12">Actividad: Crea una red poderosa!</a></span></li></ul></li><li><span><a href="#Pytorch" data-toc-modified-id="Pytorch-4">Pytorch</a></span></li></ul></div>

![](extra/atlas.png)
# Cargamos librerias

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import scipy as sc
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display
import sys

## metricas de evaluacion (sigmas) + funciones de utilidad

![http://i.imgur.com/Hflz2lG.jpg](http://i.imgur.com/Hflz2lG.jpg)

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

    
def AMSScore(s,b): 
    return np.sqrt (2.*( (s + b + 10.)*np.log(1.+s/(b+10.))-s))

def eval_model(Y_true_train,Y_pred_train,w_train,Y_true_test,Y_pred_test,w_test):
    ratio =  float(len(X_train)) /float(len(X_test))
    TruePositive_train = w_train*(Y_true_train==1.0)*(1.0/ratio)
    TrueNegative_train = w_train*(Y_true_train==0.0)*(1.0/ratio)
    TruePositive_valid = w_test*(Y_true_test==1.0)*(1.0/(1-ratio))
    TrueNegative_valid = w_test*(Y_true_test==0.0)*(1.0/(1-ratio))
    s_train = sum ( TruePositive_train*(Y_pred_train==1.0) )
    b_train = sum ( TrueNegative_train*(Y_pred_train==1.0) )
    s_test = sum ( TruePositive_valid*(Y_pred_test==1.0) )
    b_test = sum ( TrueNegative_valid*(Y_pred_test==1.0) )
    score_train = AMSScore(s_train,b_train)
    score_test = AMSScore(s_test,b_test)
    print('--- Resultados --')
    print('- AUC train: {:.3f} '.format(roc_auc_score(Y_train,Y_train_pred)))
    print('- AUC test : {:.3f} '.format(roc_auc_score(Y_test,Y_test_pred)))
    print('- AMS train: {:.3f} sigma'.format(score_train))
    print('- AMS test : {:.3f} sigma'.format(score_test))
    return score_train, score_test

def plot_roc(clf,Y_test,Y_test_prob):
    fpr, tpr, thresholds = roc_curve(Y_test, Y_test_prob)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1, alpha=0.3, label=str(clf.__class__.__name__))
    plt.plot(np.linspace(0,1,100),np.linspace(0,1,100), lw=2, alpha=0.3, label='Suerte')
    plt.legend(loc='lower right')
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.tight_layout()
    return

def keras_graph(model):
    from IPython.display import SVG
    from keras.utils.vis_utils import model_to_dot
    return SVG(model_to_dot(model).create(prog='dot', format='svg'))

## Datos de entrenamiento!

Quieres saber mas? Visita

In [ ]:
df=pd.read_csv('datos/training.csv')
print(df.shape)
df.head(1)

## preprocesamiento para X y Y

In [ ]:
Y = df['Label'].replace(to_replace=['s','b'],value=[1,0]).values
weights = df['Weight'].values
X = df.drop(['EventId','Label','Weight'],axis=1).values

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test,w_train,w_test = train_test_split(X,Y,weights,train_size=0.3)
print(X_train.shape,Y_train.shape,w_train.shape)
print(X_test.shape,Y_test.shape,w_test.shape)

# Tensorflow
[![](extra/tf_logo.jpg)](https://www.tensorflow.org/)

In [ ]:
import tensorflow as tf

# Keras

[![](extra/keras-logo.png)](https://keras.io/)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import backend as K
import keras

## Red sencillita

In [ ]:
K.clear_session()

model = Sequential()
model.add(Dense(units=10, input_dim=X_train.shape[-1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
print(model.summary())
model.fit(X_train, Y_train, epochs=5, batch_size=32)

In [ ]:
pcut=0.5
Y_train_pred = model.predict(X_train).flatten() > pcut
Y_test_pred = model.predict(X_test).flatten() > pcut
Y_train_prob= model.predict_proba(X_train).flatten()
Y_test_prob = model.predict_proba(X_test).flatten()
eval_model(Y_train,Y_train_pred,w_train,Y_test,Y_test_pred,w_test)

## Calidad de vida: liveloss!

In [ ]:
from livelossplot import PlotLossesKeras

model.fit(X_train, Y_train,
          epochs=5, batch_size=32,
          validation_data=(X_test, Y_test),
          callbacks=[PlotLossesKeras()],
          verbose=0)

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
SVG(model_to_dot(model).create(prog='dot', format='svg'))

## Calidad de vida: agregando un tensorboard
Para poder ver tu graph y metrics corre tensorboard
```
tensorboard --logdir Graph/
```
podras visualizar todo en la liga [http://localhost:6006](http://localhost:6006)

In [ ]:
 tb_call = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

model.fit(X_train, Y_train, epochs=5, batch_size=32,callbacks=[tb_call])

## Trucos: Regularizadores de redes

**L2** pesos mas "equitativos"
**L1** pesos mas "sparse", muchos 0's.

In [ ]:
from keras import regularizers

K.clear_session()

model = Sequential()
model.add(Dense(units=10, input_dim=X_train.shape[-1],
                kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),
                activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
print(model.summary())
model.fit(X_train, Y_train, epochs=5, batch_size=32)

## Actividad: Crea una red profunda

Sugerencias:
* Agrega mas capas
* Hay alguna relacion entre las dimensiones de las capas?
* Como calculas el numero de parametros a entrenar?
* Cual es el efecto del learning rate?

## trucos: Dropout

Matar conexiones de manera aleatoria puede mejorar las conexiones.

Mas info en https://keras.io/layers/core/

In [ ]:
from keras.layers import Dense, Activation, Dropout

K.clear_session()

model = Sequential()
model.add(Dense(units=10, input_dim=X_train.shape[-1], kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.2)) # 20% de las conexiones se apagan
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
print(model.summary())
model.fit(X_train, Y_train, epochs=5, batch_size=32)

## Trucos: BatchNormalization y Activacion!

Algunas activaciones https://keras.io/layers/advanced-activations/

Batch Normalization https://keras.io/layers/normalization/

In [ ]:
from keras.layers import Dense, Activation, Dropout, BatchNormalization

K.clear_session()

model = Sequential()
model.add(Dense(units=100, input_dim=X_train.shape[-1], kernel_initializer='normal'))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
print(model.summary())
model.fit(X_train, Y_train, epochs=5, batch_size=32)

## Keras Functional

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense

K.clear_session()

inputs = Input(shape=(X_train.shape[-1],))
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
pred = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=pred)

model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
print(model.summary())
model.fit(X_train, Y_train, epochs=5, batch_size=32)
pcut=0.5
Y_train_pred = model(X_train).flatten() > pcut
Y_test_pred = model(X_test).flatten() > pcut
Y_train_prob= model(X_train).flatten()
Y_test_prob = model(X_test).flatten()
eval_model(Y_train,Y_train_pred,w_train,Y_test,Y_test_pred,w_test)

## Truco: Skip connections

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Concatenate

K.clear_session()

inputs = Input(shape=(X_train.shape[-1],))
x1 = Dense(64, activation='relu')(inputs)
x2 = Dense(64, activation='relu')(x1)
skipped = Concatenate()([x1,x2])
pred = Dense(1, activation='sigmoid')(skipped)

model = Model(inputs=inputs, outputs=pred)

model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
print(model.summary())
model.fit(X_train, Y_train, epochs=5, batch_size=32)
pcut=0.5
Y_train_pred = model(X_train).flatten() > pcut
Y_test_pred = model(X_test).flatten() > pcut
Y_train_prob= model(X_train).flatten()
Y_test_prob = model(X_test).flatten()
eval_model(Y_train,Y_train_pred,w_train,Y_test,Y_test_pred,w_test)
keras_graph(model)


## Optimizador




![](extra/saddle_point_evaluation_optimizers.gif)

Imagenes de Sebastian Ruder (http://ruder.io)

![](extra/beales_20171117_00-02-20_2d.gif)

Imagenes de https://github.com/wassname/viz_torch_optim'


Y hay muchos mas en camino! (http://www.fast.ai/2018/07/02/adam-weight-decay/)

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Concatenate
from keras import optimizers

lr = 0.01
K.clear_session()
for opt in [optimizers.SGD(lr=lr),
            optimizers.SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True),
            optimizers.Adam(lr=lr),
            optimizers.Adagrad(lr=lr)
           ]:
    print('===={}==='.format(str(opt.__class__)))
    inputs = Input(shape=(X_train.shape[-1],))
    x1 = Dense(64, activation='relu')(inputs)
    x2 = Dense(64, activation='relu')(x1)
    skipped = Concatenate()([x1,x2])
    pred = Dense(1, activation='sigmoid')(skipped)
    model = Model(inputs=inputs, outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(X_train, Y_train, epochs=5, batch_size=32)
    pcut=0.5
    Y_train_pred = model(X_train).flatten() > pcut
    Y_test_pred = model(X_test).flatten() > pcut
    Y_train_prob= model(X_train).flatten()
    Y_test_prob = model(X_test).flatten()
    eval_model(Y_train,Y_train_pred,w_train,Y_test,Y_test_pred,w_test)

## Controlando el learning rate

![image.png](extra/learn_rate.png)

Imagen de [cs231n](http://cs231n.stanford.edu/)

![](extra/lr_finder.png)

![](extra/Lanscape_lr.png)

![](extra/cyclic_lr.png)

* Cyclical Learning Rates for Training Neural Networks (https://arxiv.org/abs/1506.01186)
* Setting the learning rate of your neural network. (https://www.jeremyjordan.me/nn-learning-rate/)
* SNAPSHOT ENSEMBLES: TRAIN 1, GET M FOR FREE (https://arxiv.org/pdf/1704.00109.pdf)
* SGDR: STOCHASTIC GRADIENT DESCENT WITH WARM RESTARTS (https://arxiv.org/pdf/1608.03983.pdf)

In [ ]:
from keras.callbacks import LearningRateScheduler
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))
        
def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 1.0
    lrate = initial_lrate * np.power(drop,  
           np.floor((1+epoch)/epochs_drop))
    return lrate

loss_history = LossHistory()
lrate = LearningRateScheduler(step_decay)
callbacks = [loss_history, lrate]


hist = model.fit(X_train, Y_train,
          epochs=5, batch_size=32,callbacks=callbacks)

plt.plot(loss_history.losses,loss_history.lr)
plt.xlabel('learning rate')
plt.ylabel('loss')
plt.show()

## Actividad: Crea una red poderosa!

Sugerencias:
* Usa los trucos que vimos
* Cual es el efecto de entrenar con un learning rate bajo, alto?
* Ojo con le optimizador y el learning rate, es super importante!

# Pytorch


[![](extra/pytorch-logo.png)](https://pytorch.org/)



In [ ]:
import torch
from torch.autograd import Variable

In [ ]:
class Net(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(30, 50)
        self.relu1 = nn.ReLU()
        self.dout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(50, 100)
        self.prelu = nn.PReLU(1)
        self.out = nn.Linear(100, 1)
        self.out_act = nn.Sigmoid()
        
    def forward(self, input_):
        a1 = self.fc1(input_)
        h1 = self.relu1(a1)
        dout = self.dout(h1)
        a2 = self.fc2(dout)
        h2 = self.prelu(a2)
        a3 = self.out(h2)
        y = self.out_act(a3)
        return y
    
net = Net()
opt = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [ ]:
e_losses = []
num_epochs = 5
batch_size=128
for e in range(num_epochs):
    print(e,end='..')
    losses=[]
    for beg_i in range(0, X.shape[0], batch_size):
        x_batch = torch.from_numpy(X)[beg_i:beg_i + batch_size, :]
        y_batch = torch.from_numpy(Y.reshape(-1,1))[beg_i:beg_i + batch_size, :]
        x_batch = Variable(x_batch).float()
        y_batch = Variable(y_batch).float()

        opt.zero_grad()
        # (1) Forward
        y_hat = net(x_batch)
        # (2) Compute diff
        loss = criterion(y_hat, y_batch)
        # (3) Compute gradients
        loss.backward()
        # (4) update weights
        opt.step()        
        losses.append(loss.data.numpy())
    e_losses.append(np.mean(losses))

plt.plot(e_losses)